### **Configuration**

In [1]:
# Dependencies
import os
from config import DARTMOUTH_API_KEY, DARTMOUTH_CHAT_API_KEY
from langchain_dartmouth.llms import ChatDartmouth

# Retrieving keys and creating environment variables
os.environ['DARTMOUTH_CHAT_API_KEY'] = DARTMOUTH_CHAT_API_KEY
os.environ['DARTMOUTH_API_KEY'] = DARTMOUTH_API_KEY

# Defining llm and embeddings models
llm_model_name = "openai.gpt-4o-mini-2024-07-18"
embeddings_model_name = "bge-large-en-v1-5"

# Defining keywords and sources
keywords = "Trump"
source="cnn"

### **Building Knowledge Base**

In [2]:
# Importing dependencies
import requests
import os
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
import re

# Creating directory to hold scraped articles
os.makedirs(name=f"{keywords}_{source}")

# Creating newsapi client
newsapiclient = NewsApiClient(api_key="03122dc3b7b84ea29212ca965b40c7aa")

# Querying articles
articles = newsapiclient.get_everything(q=keywords, sources=source ,page_size=1)
articles = articles['articles']

# Scraping articles and saving in directory
for article in articles:
	url = article['url']
	response = requests.get(url=url)

	# Printing article content to directory if valid response
	if response.status_code==200:
		beautifulsoup = BeautifulSoup(response.content, "html.parser")
		article_paragraphs = beautifulsoup.find_all("p")

		# Cleaning article title
		article_title = re.sub(' ', '_', article['title'])

		with open(file=f"{keywords}_{source}/{article_title}.txt", mode="w") as fp:
			for paragraph in article_paragraphs:
				fp.write(str(paragraph.get_text()))

### **Loading and Splitting Documents**

In [3]:
# Importing dependencies
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter

# Defining directory path
directory = f"{keywords}_{source}"

# Creating loader and splitter
loader = DirectoryLoader(path=directory, glob="*.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base", 
                                                       chunk_size=256, chunk_overlap=0)

# Loading and splitting documents
docs = loader.load_and_split(text_splitter=splitter)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### **Embedding and Storing Documents**

In [ ]:
# Importing dependencies
from langchain_dartmouth.embeddings import DartmouthEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Creating embeddings model
embeddings = DartmouthEmbeddings(model_name=embeddings_model_name, dartmouth_api_key=str(DARTMOUTH_API_KEY))

# Embedding documents and storing them in memory
vector_store = InMemoryVectorStore(embedding=embeddings)
_ = vector_store.add_documents(docs)

['5ce81580-4752-4ae9-b6bd-108415723856',
 'a19e3f5c-2691-402c-b53f-b1bf88cb52ab',
 'f578be28-b3e1-48cf-be57-a8c80359024a',
 '732260b6-1bc1-420f-8045-381351a00b59']

### **Retrieval**

In [5]:
query = "Is Trump good for climate?"

docs = vector_store.similarity_search(query, k=2)

### **Generation**

In [6]:
# Importing dependencies
from langchain_dartmouth.llms import ChatDartmouthCloud

# Initializing variable referencing LLM
llm = ChatDartmouthCloud(model_name=llm_model_name)

# Creating augmented prompt
prompt = (
    "Answer the following query: "
    + query
    + "\n\nBase your response on the following context: \n\n"
)

for doc in docs:
    prompt += doc.page_content + "\n--\n"

# Querying LLM and printing response
response = llm.invoke(prompt)
response.pretty_print()

================================== Ai Message ==================================

Based on the provided context, the Trump administration's actions, particularly the significant layoffs at the National Oceanic and Atmospheric Administration (NOAA), suggest a negative impact on climate-related research and forecasting. The termination of around 800 employees, including those involved in weather, climate, and environmental science, raises concerns about the government's commitment to climate science and monitoring. 

The cuts to staffing in critical areas of weather and climate prediction could undermine the ability to forecast severe weather events and understand climate trends, potentially placing public safety at risk. Therefore, it can be argued that Trump's policies and decisions, as reflected in the context, are not favorable for climate science and environmental monitoring.


### **Evaluation**